In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02
X = torch.arange(start,end, step).unsqueeze(dim=1)
y = weight*X + bias
X[:10], y[:10]

In [52]:
plt.show(X,y)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous